In [1]:
import requests
import json
import re

In [ ]:
token = ''
headers = {
    "Authorization": f"Bearer {token}"
}


# API URL
url = "http://localhost:3000/api/models"

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 檢查是否有錯誤

    # 解析 JSON 資料
    data = response.json()
    print("取得的資料：", data)

except requests.exceptions.RequestException as e:
    print("請求失敗：", e)

res = requests.post('http://127.0.0.1:5000/wait_ai_handle_list', json={'count': 10})
res_objs = json.loads(res.text)

def chat_with_model(token, content):
    url = 'http://localhost:3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {
        "model": "gemma3:12b-it-qat",
        "messages": [
            {
                "role": "system",
                "content": """
                    你是一位專業的新聞分析師，擅長從新聞提取關鍵資訊。
                    1. 標題：
                    - 整理出一個可以貫穿整個文章的標題，使用肯定句，最多35個文字。

                    2. 分類
                    - 將內容分類，並回答是屬於哪一項，政治、國際、地方、社會、娛樂、生活、氣象、健康、體育、財經、旅遊、科技。

                    3. 關鍵字：
                    - 提取最多5個核心的名詞，包含團體、政黨、名人、技術。
                    - 名人需判斷是否為政治人物，是政治人物需加上政黨名稱。
                    - 使用", "作為關鍵字的間格符號。
                    4. 語意分析：
                    - 分析內容的情感類別，並回答是正面、負面還是中立。
                    5. 將以上結果轉換成json格式
                    - {"title": "標題結果", "category": ["分類結果1", "分類結果2"], "keyword": ["關鍵字結果1", "關鍵字結果2"], "sentiment_analysis": "語意分析結果"}
                    """
            },
            {
                "role": "user",
                "content": content
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()


url = "http://127.0.0.1:5000/add_ai_news"  # 替換為實際的 API URL

for obj in res_objs[1:]:
    jsondata = chat_with_model(token, obj['news_content'])
    content1 = jsondata['choices'][0]['message']['content']
    re_obj = re.search(r'```json(.*?)```', content1, re.DOTALL)
    result_json = json.loads(re_obj[1])

    data = {
        "news_id": obj['id'],
        **result_json
    }

    response = requests.post(url, json=data)

    if response.status_code == 201:
        print("新增成功:", obj['id'], response.json())
    else:
        print("新增失敗:", response.status_code, response.json())
